In [3]:
# GENERATE TRIALS AND SEGMENT DATA 

import os
import re
import pandas as pd
import mne
from pathlib import Path
import logging

# Setup logging to file AND console
LOG_FILE = "processing.log"
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)s | %(message)s',
    handlers=[
        logging.FileHandler(LOG_FILE, mode='w'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger()

# Output directory
OUTPUT_DIR = Path("/home/jaizor/jaizor/xtra/derivatives/eeg/crop")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Dataset configuration
datasets = {
    'bima_off': {
        'data_dir': "/home/jaizor/jaizor/xtra/data/PD_Bima",
        'filename_pattern': r"PD_(\d{3})_bima_DBSOFF\.mff",
        'trials_csv': "bima_trials.csv",
        'trial_numbers': [3]  # ONLY trial 3
    },
    'rest_off': {
        'data_dir': "/home/jaizor/jaizor/xtra/data/PD_Rest/OFF",
        'filename_pattern': r"PD_(\d{3})_rest_hand_foot_DBSOFF\.mff",
        'trials_csv': "rest_trials.csv",
        'trial_numbers': [1, 2]
    }
}

def get_subjects(data_dir, pattern):
    subjects = []
    compiled = re.compile(pattern)
    for f in os.listdir(data_dir):
        if f.endswith('.mff') and compiled.match(f):
            num = int(compiled.match(f).group(1))
            subjects.append(f"sub-{num:02d}")
    return sorted(subjects, key=lambda x: int(x.split('-')[1]))

def process_dataset(dataset_key, config):
    logger.info(f"{'='*60}")
    logger.info(f"STARTING DATASET: {dataset_key}")
    logger.info(f"{'='*60}")
    
    # Load trials
    if not os.path.exists(config['trials_csv']):
        logger.error(f"Trials file missing: {config['trials_csv']}")
        return 0
    
    trials_df = pd.read_csv(config['trials_csv'])
    subjects = get_subjects(config['data_dir'], config['filename_pattern'])
    logger.info(f"Found {len(subjects)} subjects: {subjects}")
    
    saved_count = 0
    for sub in subjects:
        logger.info(f"Processing {sub}...")
        
        # Find .mff file
        subj_num = sub.split('-')[1]
        mff_files = [f for f in os.listdir(config['data_dir']) 
                     if f.endswith('.mff') and subj_num in f]
        if not mff_files:
            logger.warning(f"No .mff file found for {sub}")
            continue
        
        try:
            raw = mne.io.read_raw_egi(
                os.path.join(config['data_dir'], mff_files[0]),
                preload=True, verbose=False
            )
        except Exception as e:
            logger.error(f"Failed to load {mff_files[0]}: {e}")
            continue
        
        # 🔍 CONFIRM: Log stim channel count BEFORE filtering
        stim_chs_before = [ch for ch, t in zip(raw.ch_names, raw.get_channel_types()) if t in ['stim', 'misc']]
        logger.debug(f"Original stim/misc channels: {stim_chs_before}")
        
        # Filter ONLY EEG channels — stim/misc are preserved untouched
        raw_f = raw.copy().filter(
            l_freq=1.0, h_freq=100.0,
            picks=['eeg'], method='fir', phase='zero',
            fir_window='hamming', fir_design='firwin',
            n_jobs=1, verbose=False
        )
        
        # 🔍 CONFIRM: stim channels still present after filtering
        stim_chs_after = [ch for ch, t in zip(raw_f.ch_names, raw_f.get_channel_types()) if t in ['stim', 'misc']]
        if len(stim_chs_after) != len(stim_chs_before):
            logger.warning(f"Stim channel count changed! Before: {len(stim_chs_before)}, After: {len(stim_chs_after)}")
        else:
            logger.debug(f"Stim channels preserved: {len(stim_chs_after)} channels")
        
        # Get requested trials
        sub_trials = trials_df[
            (trials_df['subject'] == sub) &
            (trials_df['trial_number'].isin(config['trial_numbers']))
        ].sort_values('trial_number')
        
        if sub_trials.empty:
            logger.warning(f"No matching trials for {sub}")
            continue
        
        out_subdir = OUTPUT_DIR / dataset_key / sub
        out_subdir.mkdir(parents=True, exist_ok=True)
        
        for _, row in sub_trials.iterrows():
            tmin, tmax_expected = row['start_time'], row['end_time']
            expected_duration = row.get('duration_s', tmax_expected - tmin)
            
            # Safety: clamp tmax to recording end
            tmax = min(tmax_expected, raw_f.times[-1])
            if tmin >= raw_f.times[-1]:
                logger.warning(f"Trial {row['trial_number']} starts after recording ends. Skipped.")
                continue
            
            try:
                cropped = raw_f.copy().crop(tmin=tmin, tmax=tmax, include_tmax=True)
                actual_duration = cropped.times[-1] - cropped.times[0]
                
                # 🔍 VALIDATION: Check duration deviation
                duration_diff = abs(actual_duration - expected_duration)
                if duration_diff > 2.0:  # more than 2 seconds off
                    logger.warning(
                        f"Trial {row['trial_number']} duration mismatch: "
                        f"expected={expected_duration:.2f}s, actual={actual_duration:.2f}s "
                        f"(Δ={duration_diff:.2f}s)"
                    )
                
                # ✅ FIXED: Use MNE-compliant filename (_eeg.fif)
                fname = out_subdir / f"{dataset_key}_{sub}_trial{row['trial_number']:02d}_eeg.fif"
                cropped.save(fname, overwrite=True)
                
                # Log stim info for this cropped segment
                stim_in_cropped = sum(1 for t in cropped.get_channel_types() if t in ['stim', 'misc'])
                logger.info(
                    f"Saved trial {row['trial_number']}: "
                    f"{tmin:.2f}–{tmax:.2f}s | actual={actual_duration:.2f}s | "
                    f"stim_channels={stim_in_cropped} | file={fname.name}"
                )
                saved_count += 1
                
            except Exception as e:
                logger.error(f"Failed to crop/save trial {row['trial_number']} for {sub}: {e}")
    
    logger.info(f"✅ Finished {dataset_key}: {saved_count} trials saved.")
    return saved_count


In [4]:
# RUN ALL DATASETS AND GENERATE SUMMARY
if __name__ == "__main__":
    summary = {}
    total_saved = 0
    
    for name, cfg in datasets.items():
        count = process_dataset(name, cfg)
        summary[name] = count
        total_saved += count
    
    # Final summary
    logger.info("\n" + "="*60)
    logger.info("PROCESSING COMPLETE — SUMMARY")
    logger.info("="*60)
    for ds, count in summary.items():
        logger.info(f"{ds:12} : {count} trials saved")
    logger.info(f"{'TOTAL':12} : {total_saved} trials saved")
    logger.info(f"All files saved to: {OUTPUT_DIR}")
    logger.info(f"Full log available at: {LOG_FILE}")

2025-10-08 14:12:14,900 | INFO | ============================================================
2025-10-08 14:12:14,902 | INFO | STARTING DATASET: bima_off
2025-10-08 14:12:14,903 | INFO | ============================================================
2025-10-08 14:12:14,906 | INFO | Found 12 subjects: ['sub-01', 'sub-02', 'sub-03', 'sub-05', 'sub-06', 'sub-07', 'sub-08', 'sub-09', 'sub-10', 'sub-11', 'sub-12', 'sub-14']
2025-10-08 14:12:14,906 | INFO | Processing sub-01...


Writing /home/jaizor/jaizor/xtra/derivatives/eeg/crop/bima_off/sub-01/bima_off_sub-01_trial03_eeg.fif
Closing /home/jaizor/jaizor/xtra/derivatives/eeg/crop/bima_off/sub-01/bima_off_sub-01_trial03_eeg.fif
[done]


2025-10-08 14:12:19,449 | INFO | Saved trial 3: 339.87–377.21s | actual=37.33s | stim_channels=10 | file=bima_off_sub-01_trial03_eeg.fif
2025-10-08 14:12:19,449 | INFO | Processing sub-02...


Writing /home/jaizor/jaizor/xtra/derivatives/eeg/crop/bima_off/sub-02/bima_off_sub-02_trial03_eeg.fif
Closing /home/jaizor/jaizor/xtra/derivatives/eeg/crop/bima_off/sub-02/bima_off_sub-02_trial03_eeg.fif
[done]


2025-10-08 14:12:22,836 | INFO | Saved trial 3: 218.08–261.74s | actual=43.67s | stim_channels=10 | file=bima_off_sub-02_trial03_eeg.fif
2025-10-08 14:12:22,837 | INFO | Processing sub-03...


...

2025-10-08 14:13:22,568 | INFO | Saved trial 2: 205.04–265.04s | actual=60.00s | stim_channels=8 | file=rest_off_sub-12_trial02_eeg.fif
2025-10-08 14:13:22,569 | INFO | Processing sub-14...


Writing /home/jaizor/jaizor/xtra/derivatives/eeg/crop/rest_off/sub-14/rest_off_sub-14_trial01_eeg.fif
Closing /home/jaizor/jaizor/xtra/derivatives/eeg/crop/rest_off/sub-14/rest_off_sub-14_trial01_eeg.fif
[done]


2025-10-08 14:13:25,937 | INFO | Saved trial 1: 118.69–178.69s | actual=60.00s | stim_channels=7 | file=rest_off_sub-14_trial01_eeg.fif


Writing /home/jaizor/jaizor/xtra/derivatives/eeg/crop/rest_off/sub-14/rest_off_sub-14_trial02_eeg.fif
Closing /home/jaizor/jaizor/xtra/derivatives/eeg/crop/rest_off/sub-14/rest_off_sub-14_trial02_eeg.fif
[done]


2025-10-08 14:13:26,235 | INFO | Saved trial 2: 188.13–248.13s | actual=60.00s | stim_channels=7 | file=rest_off_sub-14_trial02_eeg.fif
2025-10-08 14:13:26,236 | INFO | ✅ Finished rest_off: 24 trials saved.
2025-10-08 14:13:26,279 | INFO | 
2025-10-08 14:13:26,279 | INFO | PROCESSING COMPLETE — SUMMARY
2025-10-08 14:13:26,280 | INFO | ============================================================
2025-10-08 14:13:26,280 | INFO | bima_off     : 12 trials saved
2025-10-08 14:13:26,281 | INFO | rest_off     : 24 trials saved
2025-10-08 14:13:26,282 | INFO | TOTAL        : 36 trials saved
2025-10-08 14:13:26,282 | INFO | All files saved to: /home/jaizor/jaizor/xtra/derivatives/eeg/crop
2025-10-08 14:13:26,282 | INFO | Full log available at: processing.log
